In [1]:
import os
import re
import pandas as pd

In [4]:
results_folder = [os.pardir,"Benchmark results","halloc_opts"]
# results_folder = [os.pardir,"Benchmark results","halloc_opts","halloc_results_crash"]
# results_folder = [os.pardir,"Benchmark results"]
print(os.path.join(*results_folder))


../Benchmark results/halloc_opts


In [5]:
parsable_keys = {
    "HALLOC_FRACTION": "hf-",
    "BUSY_FRACTION": "bu-",
    "ROOMY_FRACTION": "ro-",
    "SPARSE_FRACTION": "sp-",
    "MAX_BLOCK_SZ": "mb-",
    "MAX_NSIZES": "ns-",
    "MAX_NCHUNK_IDS": "nc-",
    "BLOCK_STEP": "bst-",
    "NUNITS": "nu-",
    "MIN_BLOCK_SZ": "mibs-",
    "MAX_BLOCK_SZ2": "mabs-",
}

In [6]:
default_params = {
    "HALLOC_FRACTION": 0.25,
    "BUSY_FRACTION": 0.835,
    "ROOMY_FRACTION": 0.6,
    "SPARSE_FRACTION": 0.012,
    "MAX_BLOCK_SZ": 3072,
    "MAX_NSIZES": 16,
    "MAX_NCHUNK_IDS": 8,
    "BLOCK_STEP": 16,
    "NUNITS": 8,
    "MIN_BLOCK_SZ": 16,
}

In [7]:
expr = r'total\s+images\/sec:\s+(\d+\.*\d*)'
r = re.compile(expr)

In [8]:
cols = ["ALLOCATOR", "NETWORK", "IMAGES_SEC", "PROFILING"] + list(default_params.keys())
df = pd.DataFrame(columns=cols)
print(df)

Empty DataFrame
Columns: [ALLOCATOR, NETWORK, IMAGES_SEC, PROFILING, HALLOC_FRACTION, BUSY_FRACTION, ROOMY_FRACTION, SPARSE_FRACTION, MAX_BLOCK_SZ, MAX_NSIZES, MAX_NCHUNK_IDS, BLOCK_STEP, NUNITS, MIN_BLOCK_SZ]
Index: []


In [9]:
for file in os.listdir(os.path.join(*results_folder)):
    # print(file)
    file_name = os.path.splitext(file)
    if file_name[-1] == ".txt":
        params = default_params.copy()
        # print(file_name[0].split('_'))

        file_split = file_name[0].split("_")
        params["ALLOCATOR"] = file_split.pop(0)
        params["NETWORK"] = file_split.pop(0)
        params["PROFILING"] = "prof" in file_split
        for part in file_split:
            for key, value in parsable_keys.items():
                if value in part:
                    # print(key, part.split('-')[-1])
                    if key == "MAX_BLOCK_SZ2":
                        key = "MAX_BLOCK_SZ"
                    params[key] = float(part.split("-")[-1])
        # print(params)

        matches = [
            re.findall(expr, line) for line in open(os.path.join(*results_folder, file))
        ]
        matches = [float(x[0]) for x in matches if len(x)]
        # boardprint(matches)

        if len(matches):
            for match in matches:
                params["IMAGES_SEC"] = match
                df = df.append(params, ignore_index=True)
        else:
            params["IMAGES_SEC"] = float("NaN")
            df = df.append(params, ignore_index=True)
df

,ALLOCATOR,NETWORK,IMAGES_SEC,PROFILING,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION,MAX_BLOCK_SZ,MAX_NSIZES,MAX_NCHUNK_IDS,BLOCK_STEP,NUNITS,MIN_BLOCK_SZ
0,halloc,alex,36.16,False,0.25,0.835,0.60,0.200,3072,16,8,16,8,16
1,halloc,resnet,NaN,False,0.25,0.700,0.60,0.012,3072,16,8,16,8,16
2,halloc,resnet,NaN,False,0.25,0.835,0.80,0.012,3072,16,8,16,8,16
3,halloc,resnet,NaN,False,0.25,0.900,0.60,0.012,3072,16,8,16,8,16
4,halloc,resnet,NaN,False,0.70,0.835,0.60,0.012,3072,16,8,16,8,16
5,halloc,resnet,NaN,False,0.25,0.835,0.75,0.012,3072,16,8,16,8,16
6,halloc,alex,NaN,False,0.25,0.835,0.60,0.012,3072,2,8,16,8,16
7,halloc,resnet,NaN,False,0.25,0.835,0.60,0.012,3072,16,2,16,8,16
8,halloc,alex,NaN,False,0.25,0.835,0.60,0.012,16,16,8,16,8,16
9,halloc,resnet,NaN,False,0.25,0.750,0.60,0.012,3072,16,8,16,8,16


In [10]:
df.to_csv(os.path.join(*results_folder,'results.csv'))

In [11]:
df.describe()

,IMAGES_SEC,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION
count,65.000000,202.000000,202.000000,202.000000,202.000000
mean,36.069692,0.277228,0.833267,0.603465,0.021703
std,0.209165,0.130541,0.017512,0.029302,0.034487
min,34.910000,0.000000,0.700000,0.500000,0.012000
25%,35.990000,0.250000,0.835000,0.600000,0.012000
50%,36.050000,0.250000,0.835000,0.600000,0.012000
75%,36.200000,0.250000,0.835000,0.600000,0.012000
max,36.380000,1.000000,0.900000,0.800000,0.200000


In [19]:
df[((df.NETWORK == "alex") & (df.SPARSE_FRACTION != default_params["SPARSE_FRACTION"]))]

,ALLOCATOR,NETWORK,IMAGES_SEC,PROFILING,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION,MAX_BLOCK_SZ,MAX_NSIZES,MAX_NCHUNK_IDS,BLOCK_STEP,NUNITS,MIN_BLOCK_SZ
0,halloc,alex,36.16,False,0.25,0.835,0.6,0.20,3072,16,8,16,8,16
11,halloc,alex,36.22,False,0.25,0.835,0.6,0.14,3072,16,8,16,8,16
15,halloc,alex,36.23,False,0.25,0.835,0.6,0.04,3072,16,8,16,8,16
16,halloc,alex,36.31,False,0.25,0.835,0.6,0.18,3072,16,8,16,8,16
26,halloc,alex,36.22,False,0.25,0.835,0.6,0.06,3072,16,8,16,8,16
55,halloc,alex,36.19,False,0.25,0.835,0.6,0.08,3072,16,8,16,8,16
104,halloc,alex,36.21,False,0.25,0.835,0.6,0.02,3072,16,8,16,8,16
118,halloc,alex,36.17,False,0.25,0.835,0.6,0.12,3072,16,8,16,8,16
134,halloc,alex,36.29,False,0.25,0.835,0.6,0.16,3072,16,8,16,8,16
142,halloc,alex,36.15,False,0.25,0.835,0.6,0.10,3072,16,8,16,8,16


In [13]:
df[df.NETWORK == "alex"].describe()

,IMAGES_SEC,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION
count,65.000000,104.000000,104.000000,104.000000,104.000000
mean,36.069692,0.276442,0.833317,0.603365,0.021423
std,0.209165,0.129029,0.017301,0.028950,0.034105
min,34.910000,0.000000,0.700000,0.500000,0.012000
25%,35.990000,0.250000,0.835000,0.600000,0.012000
50%,36.050000,0.250000,0.835000,0.600000,0.012000
75%,36.200000,0.250000,0.835000,0.600000,0.012000
max,36.380000,1.000000,0.900000,0.800000,0.200000


In [18]:
from sklearn import linear_model
from numpy import isnan

# df2 = df[df.NETWORK == "alex"]
df2 = df[((df.NETWORK == "alex") & (~isnan(df.IMAGES_SEC)))]

reg = linear_model.LinearRegression()
reg.fit(df2[["PROFILING", "HALLOC_FRACTION", "BUSY_FRACTION", "ROOMY_FRACTION", "SPARSE_FRACTION", "MAX_BLOCK_SZ", "MAX_NSIZES", "MAX_NCHUNK_IDS", "BLOCK_STEP", "NUNITS", "MIN_BLOCK_SZ"]], df2["IMAGES_SEC"])

print(reg.coef_)
# print(reg.intercept_)

[-1.16671784e+00 -1.70614861e-01  1.88775013e-01  1.73128634e-01
  1.09436233e+00 -7.81035819e-05 -7.11853847e-04 -2.40571750e-03
  1.80608350e-04  0.00000000e+00  0.00000000e+00]
